In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras

print(tf.__version__)
print(keras.__version__)
print('Using GPU: ', tf.test.is_gpu_available())

1.12.0
2.1.6-tf
Using GPU:  True


In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [12]:
import tensorflow.keras.applications.resnet50 as resnet50 
import tensorflow.keras.applications.vgg19 as vgg19
import tensorflow.keras.applications.densenet as densenet
import tensorflow.keras.applications.inception_v3 as inception_v3

In [13]:
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.losses as losses
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.backend as K

# Datasets

In [3]:
from scipy.io import loadmat
from os.path import join

prefix = './Stanford_Dog_Breed'
imgfix = 'Images'
anofix = 'Annotation'

### Read the Testing Dataset

In [9]:
test = loadmat(join(prefix, 'test_list.mat'))

ftest = test['file_list']
ltest = test['labels']
ntest = ltest.shape[0]

print('Size of the testing set: \t', ntest)

Size of the testing set: 	 8580


### Catagory 2 Index Table

In [10]:
DogBreed = {}
for i in range(1, ntrain):
    l = ltrain[i][0]
    c = ftrain[i][0][0].split('/')[0]
    c = c.split('-')[1]
    if c not in DogBreed:
        DogBreed[c] = l

### Helper Functions

In [22]:
def gen_path(idx):
    try:    path = join(prefix, imgfix, 'Test', ftest[idx][0][0])
    except: path = join(prefix, imgfix, 'Test', ftest[-1][0][0])      
    return  path
def gen_labl(idx):
    try:    labl = ltest[idx][0]
    except: labl = ltest[-1][0]
    return  labl

In [36]:
def TestDataGen(preprocess_input):
    for idx in range(ntest):
        path = gen_path(idx)
        Y = gen_labl(idx)
        img = image.load_img(gen_path(idx), target_size=(224, 224))
        X = image.img_to_array(img)
        X = np.expand_dims(X, axis=0)
        X = preprocess_input(X)
        yield X, Y

In [99]:
def DecodePred(Pred, Y):
    cor, num, top = 0, len(Pred), len(Pred[0])
    for idx in range(num):
        pred, y = Pred[idx], Y[idx]
        for k in range(top):
            lbl = pred[k][1]
            if lbl not in DogBreed: continue
            elif y==DogBreed[lbl]: cor+=1; break
    return cor / num

# Evaluate the Performance of Naive Models

### Inception V3

In [92]:
model1 = inception_v3.InceptionV3(weights='imagenet', include_top=True)

In [93]:
model1.compile(optimizer='SGD',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [94]:
Pred1 = model1.predict_generator(
    TestDataGen(inception_v3.preprocess_input),
    steps=ntest,
    verbose=1, 
    use_multiprocessing=True)

Y = [gen_labl(idx) for idx in range(ntest)]            


8580/8580 [==============================] - 166s 19ms/step


In [107]:
[DecodePred(inception_v3.decode_predictions(Pred1, top), Y) for top in range(1,20)]

[0.7813519813519814,
 0.8752913752913752,
 0.9059440559440559,
 0.9180652680652681,
 0.9247086247086247,
 0.9280885780885781,
 0.9311188811188811,
 0.9334498834498834,
 0.9356643356643357,
 0.9372960372960373,
 0.9384615384615385,
 0.9398601398601398,
 0.9406759906759907,
 0.9416083916083916,
 0.9418414918414918,
 0.9423076923076923,
 0.9425407925407926,
 0.9427738927738928,
 0.943006993006993]

### VGG19

In [103]:
model2 = vgg19.VGG19(weights='imagenet', include_top=True)

574717952/574710816 [==============================] - 1996s 3us/step


In [104]:
model2.compile(optimizer='SGD',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [105]:
Pred2 = model2.predict_generator(
    TestDataGen(vgg19.preprocess_input),
    steps=ntest,verbose=1, 
    use_multiprocessing=True)

Y = [gen_labl(idx) for idx in range(ntest)]       


8580/8580 [==============================] - 190s 22ms/step


In [108]:
[DecodePred(vgg19.decode_predictions(Pred2, top), Y) for top in range(1,20)]

[0.7132867132867133,
 0.83006993006993,
 0.8735431235431236,
 0.8931235431235431,
 0.905011655011655,
 0.9129370629370629,
 0.91993006993007,
 0.9243589743589744,
 0.9280885780885781,
 0.9301864801864802,
 0.9318181818181818,
 0.9339160839160839,
 0.9348484848484848,
 0.9356643356643357,
 0.9362470862470863,
 0.9371794871794872,
 0.9382284382284383,
 0.938927738927739,
 0.9397435897435897]

### DenseNet121

In [86]:
model3 = densenet.DenseNet121(weights='imagenet', include_top=True)

33193984/33188688 [==============================] - 105s 3us/step


In [87]:
model3.compile(optimizer='SGD',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

In [101]:
Pred3 = model3.predict_generator(
    TestDataGen(densenet.preprocess_input),
    steps=ntest,
    verbose=1, 
    use_multiprocessing=True)

Y = [gen_labl(idx) for idx in range(ntest)]   


8580/8580 [==============================] - 125s 15ms/step


In [109]:
[DecodePred(densenet.decode_predictions(Pred3, top), Y) for top in range(1,20)]

[0.7997668997668997,
 0.8846153846153846,
 0.9118881118881119,
 0.9233100233100233,
 0.9304195804195804,
 0.9346153846153846,
 0.9382284382284383,
 0.9402097902097902,
 0.9416083916083916,
 0.9421911421911422,
 0.9427738927738928,
 0.9434731934731935,
 0.9435897435897436,
 0.9438228438228439,
 0.9444055944055944,
 0.9449883449883449,
 0.9455710955710955,
 0.9455710955710955,
 0.9456876456876457]